# Trustwise SDK Guide

This notebook provides a step-by-step guide to using the Trustwise SDK for evaluating AI-generated content with Trustwise's Safety and Alignment metrics.

## 1. Installation and Setup

First, let's install the Trustwise SDK and set up our environment.

In [ ]:
!pip install trustwise

#### Load Env vars

Load env vars from `.env`

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access environment variables
api_key = os.environ.get("TW_API_KEY")
assert api_key is not None, "TW_API_KEY is not set"

## 2. Basic Setup

Let's initialize the SDK with your API key and create a basic configuration.

In [5]:
import os
from trustwise.sdk import TrustwiseSDK
from trustwise.sdk.config import TrustwiseConfig

# Initialize the SDK
config = TrustwiseConfig(api_key=os.environ["TW_API_KEY"], base_url="https://api.trustwise.ai")
# config = TrustwiseConfig())
trustwise = TrustwiseSDK(config)

ImportError: cannot import name 'TrustwiseClientProtocol' from 'trustwise.sdk.client' (/Users/mk/work/github/trustwise/src/trustwise/sdk/client.py)

## 3. Safety Metrics

Let's explore some of the safety metrics available in the SDK.

In [17]:
# RAG SECTION
# Context -> Qdrant
# Response -> LLM
# Query -> User

# Example context for evaluation
context = [{
    "node_text": "Paris is the capital of France.",
    "node_score": 0.95,
    "node_id": "doc:idx:1"
}]

# Metrics calls post RAG

# Evaluate faithfulness
result = trustwise.safety.faithfulness.evaluate(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
    context=context
)
print(f"Faithfulness score: {result['score']}")

# Evaluate PII detection
pii_result = trustwise.safety.pii.evaluate(
    text="My email is john@example.com and my phone is 123-456-7890",
    allowlist=["john@example.com"]  # Allow specific PII
)
print(f"PII detection result: {pii_result}")

# # Feedback:
# 1. Getting query, context, response is not intuitive, do we need a helper function?
# 2. Context schema - is it only qdrant or it supports other formats?
# 3. End-end RAG example using the SDK? Example with context, query, response, etc.


Faithfulness score: 99.971924
PII detection result: {'identified_pii': [{'interval': [45, 57], 'string': '123-456-7890', 'category': 'phone number'}]}


## 4. Alignment Metrics

Now let's look at some alignment metrics to evaluate the quality of responses.

In [18]:
# Evaluate clarity
clarity_result = trustwise.alignment.v1.clarity.evaluate(
    query="What is the capital of France?",
    response="The capital of France is Paris."
)
print(f"Clarity score: {clarity_result['score']}")

# Evaluate tone
tone_result = trustwise.alignment.v1.tone.evaluate(
    response="The capital of France is Paris."
)
print(f"Tone result: {tone_result}")

Clarity score: 73.84502
Tone result: {'labels': ['neutral', 'happiness', 'realization'], 'scores': [89.106514, 6.6293826, 3.538071]}


## 5. Guardrails (Beta)

Let's create a guardrail to enforce multiple metrics at once.

In [4]:
# Create a multi-metric guardrail
guardrail = trustwise.guardrails(
    thresholds={
        "faithfulness": 100,
        "answer_relevancy": 0.7,
        "clarity": 0.7
    },
    block_on_failure=True
)

# Evaluate with multiple metrics
evaluation = guardrail.evaluate_response(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
    context=context
)

print("Guardrail Evaluation:")
print(f"Passed all checks: {evaluation['passed']}")
print(f"Response blocked: {evaluation['blocked']}")
for metric, result in evaluation['results'].items():
    print(f" - {metric}: {result['passed']} (score: {result['result'].get('score')})")

Guardrail Evaluation:
Passed all checks: False
Response blocked: True
 - faithfulness: False (score: 99.971924)


## 6. API Versioning

The SDK supports versioning for different metrics. Let's see how to work with versions.

In [6]:
# Get current versions
versions = trustwise.get_versions()
print(f"Default versions: {versions}")

# Set default version
trustwise.safety.set_version("v3")
print(f"Safety version: {trustwise.safety.version}")

# Using different version access methods
result1 = trustwise.safety.v3.faithfulness.evaluate(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
    context=context
)
result2 = trustwise.safety.faithfulness.evaluate(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
    context=context
)  # Uses default v3
print(f"Scores identical: {result1['score'] == result2['score']}")


Default versions: {'safety': ['v3'], 'alignment': ['v1']}
Safety version: v3
Scores identical: True


## 7. Next Steps

Now that you've completed this quickstart guide, you can:

1. Explore more safety metrics like answer relevancy and context relevancy
2. Try additional alignment metrics like helpfulness and formality
3. Create custom guardrails with different threshold combinations
4. Experiment with different API versions

For more detailed information, refer to the [Trustwise SDK documentation](https://supreme-happiness-lrmyw2m.pages.github.io/).